In [ ]:
import numpy as np
import sympy as sym
import matplotlib.pyplot as plt

In [ ]:
import sys
sys.path.append("..")

from ggqpy import construct_Chebyshev_quadratures
from ggqpy.functionfamiliy import Interval, FunctionFamily
from ggqpy.testproblems import example_problem, gen_poly_and_sing
from ggqpy.discretize import Discretizer
from ggqpy.compress import compress_sequence_of_functions, visualise_diagonal_dropoff, construct_A_matrix, interp_legendre
from ggqpy.optimize import QuadOptimizer
from ggqpy.visualize import plot_points

In [ ]:
x,w = np.polynomial.legendre.leggauss(10)
plt.stem(x,w)
f = lambda x: x**19 + 1
f(x)@w

In [ ]:
eps_disc = 1e-10
eps_comp = 1e2*eps_disc
eps_quad = 1e-10
I = Interval(1e-9,1)
seed = 0

In [ ]:
rng_gen = np.random.default_rng(seed)
F = example_problem(I, number_of_functions = 100, expr_gen=gen_poly_and_sing)
ex_f, ex_f_expr = F.generate_example_function()

In [ ]:
print("Example function from function space")
display(ex_f_expr)
xx = np.linspace(I.a,I.b,1000)
functions_to_plot = 5
plt.subplot(1, 2, 1)
plt.title("Example functions")
plt.plot(xx,ex_f(xx))
plt.subplot(1, 2, 2);
xx2 = np.linspace((I.a + I.b)/4,I.b,1000)
plt.plot(xx2,ex_f(xx2));

Discretization and compression

In [ ]:
disc = Discretizer(eps_disc,min_length=1e-7, interpolation_degree=15)
x_disc, w_disc, endpoints, intervals = disc.adaptive_discretization(F)
U_disc, rank = compress_sequence_of_functions(F.functions, x_disc, w_disc, eps_comp)
u_family = interp_legendre(U_disc, endpoints)

In [ ]:
print("Functions before compression:", len(F.functions))
print("Functions after compression:", u_family.number_of_functions)
A = construct_A_matrix(x_disc,w_disc,F.functions)
visualise_diagonal_dropoff(A, eps_comp)

Chebyshev Rule

In [ ]:
(x_cheb,), w_cheb = construct_Chebyshev_quadratures((x_disc,),w_disc,U_disc)

In [ ]:
print("Number of points in original discretization:", len(x_disc))
print("Number of points in Chebyshev rule:", len(x_cheb))
print("Absolute integral error for chebyshev:", abs(F.target_integral(ex_f_expr)-ex_f(x_cheb)@w_cheb))

Nonlinear optimization

In [ ]:
r = U_disc.T@w_disc

In [ ]:
opt = QuadOptimizer(u_family,r, ftol=1e-15,verbose=False)
x,w = opt.point_reduction(x_cheb, w_cheb, eps_quad)

In [ ]:
print("Quadrature length: ", len(x))
print("Absolute error:", abs(F.target_integral(ex_f_expr)-ex_f(x)@w))

Comparison with Gauss-Legendre

In [ ]:
fig, ax = plt.subplots()
x_gl,w_gl = np.polynomial.legendre.leggauss(len(x))
w_gl = w_gl*0.5*I.length()
x_gl = I.translate(x_gl)
ax.axhline(y=0, c="orange")
ax.stem(x_gl, w_gl, markerfmt = 'bo', basefmt="orange", label=r"Gauss-Legendre nodes")
ax.stem(x, w, markerfmt = 'go', basefmt="orange", label=r"New quadrature nodes")
ax.legend()
print("Gauss-Legendre",np.around(x_gl,3))
print("New quadrature",np.around(sorted(x),3))

In [ ]:
symx = sym.Symbol("x",real=True)
expr = symx**2
display(expr)
f = sym.lambdify(symx, expr, "numpy")
print("Gauss-Legendre error", abs(F.target_integral(expr) - f(x_gl)@w_gl))
print("New quadrature error", abs(F.target_integral(expr) - f(x)@w))


In [ ]:
expr = 1/symx + symx**2
display(expr)
f = sym.lambdify(symx, expr, "numpy")
print("Gauss-Legendre error", abs(F.target_integral(expr) - f(x_gl)@w_gl))
print("New quadrature error", abs(F.target_integral(expr) - f(x)@w))